#  Coursera Applied Data Science Capstone Project

### This notebook is to be used for the Coursera Capstone Project

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto
First I define the URL and read it into a dataframe

In [2]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
page  = requests.get(url)
page

<Response [200]>

In [5]:
data=page.text
soup = BeautifulSoup(data, 'html.parser')
#print(soup.prettify())
df=pd.read_html(url)
df=df[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


I then remove the rows which contain a "Not assigned" value in the Brorough column

In [6]:
#Pre-processing
#Removing "Not Assigned" Boroughs from dataframe
df=df[~df['Borough'].isin(["Not assigned"])]
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


I group my dataframe by Postcode and joining the Neighborhoods (separated by a comma) 

In [7]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood']= df['Neighbourhood'].str.join(', ')
df

,Postcode,Borough,Neighbourhood
0,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
1,M4A,North York,Victoria Village
2,M2R,North York,Willowdale West
3,M5C,Downtown Toronto,St. James Town
4,M3J,North York,"Northwood Park, York University"
...,...,...,...
98,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
99,M4N,Central Toronto,Lawrence Park
100,M4K,East Toronto,"The Danforth West, Riverdale"
101,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"


If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [8]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
1,M4A,North York,Victoria Village
2,M2R,North York,Willowdale West
3,M5C,Downtown Toronto,St. James Town
4,M3J,North York,"Northwood Park, York University"


In [9]:
df.shape

(103, 3)

## Then I use the csv file provided (since the geocoder package is very unreliable) to create the dataframe that includes the latitude and the longitude

In [10]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [11]:
df_geo.rename(columns={'Postal Code':'Postcode'},inplace=True)
results = pd.merge(df,df_geo[['Postcode','Latitude','Longitude']],on='Postcode')
results

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M2R,North York,Willowdale West,43.782736,-79.442259
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
...,...,...,...,...,...
98,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
99,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
100,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
101,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
